# Project — Data Acquisition and Ingestion
Owner: Kushal Mamillapalli
Date: 08/20

## Goals
- Pull Financial Data from Kaggle using Python code that gives a df and convert to csv

### Kept the other code just in case, but the Two cells with Kaggle are used for acquisition along with the utilities


## Utilities
Helper functions for timestamps, validation, and IO. Prefer explicit schema checks at ingestion time.

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import datetime as dt
import os

def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Kaggle Dataset Download
- Source: Kaggle API via `kagglehub`
- Target: Credit Card Fraud Detection dataset (mlg-ulb/creditcardfraud)
- Notes: Download latest version; dataset includes 285K+ transactions with 31 features for fraud classification.


In [ ]:

# Load the latest version
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "mlg-ulb/creditcardfraud",
    "creditcard.csv"
)

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# Save to your existing RAW directory
timestamp = dt.datetime.now().strftime('%Y%m%d-%H%M%S')
output_path = RAW / f"fraud_kaggle_creditcard_{timestamp}.csv"
df.to_csv(output_path, index=False)
print(f"Saved to: {output_path}")

Dataset shape: (284807, 31)
Columns: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
Saved to: ../data/raw/fraud_kaggle_creditcard_20250827-200653.csv


In [3]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('../data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## API Pull
- Source: Alpha Vantage TIME_SERIES_DAILY (fallback: yfinance)
- Notes: respect rate limits; persist only `date` and `close` for free-tier parity; cast dtypes explicitly.

In [11]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    #adjusted doesn't seem to work, premium endpoint instead of free;
    # params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df1 = pd.DataFrame(js[key]).T.reset_index()
    print('df1 columns:', df1.columns.tolist())
    #less columns in the free endpoint, had to remove some
    #df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date',"4. close":"close"})[['date','close']]
    df_api['date'] = pd.to_datetime(df_api['date'])
    #df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
    #replace with free verison
    df_api['close'] = pd.to_numeric(df_api['close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']
#adjust for close
# v_api = validate(df_api, ['date','adj_close']); v_api
v_api = validate(df_api, ['date','close']); v_api

df1 columns: ['index', '1. open', '2. high', '3. low', '4. close', '5. volume']


{'missing': [], 'shape': (100, 2), 'na_total': 0}

In [12]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_source-alpha_symbol-AAPL_20250820-114423.csv


## Web Scrape
- Target: permitted public table (e.g., Yahoo Finance Gainers)
- Risks: layout and selector drift; inconsistent number formats; robots and legal constraints.
- Mitigation: user-agent header, numeric coercion, capture full raw HTML when parsing fails.

In [13]:
# SCRAPE_URL = 'https://example.com/markets-table'  # TODO: replace with permitted page
SCRAPE_URL = 'https://finance.yahoo.com/gainers'  # TODO: replace with permitted page
#good website for gaining companies by yahoo which is one of the originals
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
    print(data)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

[['NEGG', 'Newegg Commerce, Inc.', '', '110.47+21.07(+23.57%)', '+21.07', '+23.57%', '847,086', '1.136M', '2.152B', '--', '+419.77%', ''], ['LNZA', 'LanzaTech Global, Inc.', '', '23.21+3.49(+17.70%)', '+3.49', '+17.70%', '45,606', '28,794', '53.839M', '--', '-88.26%', ''], ['SRRK', 'Scholar Rock Holding Corporation', '', '33.84+3.73(+12.40%)', '+3.73', '+12.40%', '1.791M', '1.417M', '3.253B', '--', '+225.27%', ''], ['VKTX', 'Viking Therapeutics, Inc.', '', '26.02+1.66(+6.81%)', '+1.66', '+6.81%', '14.5M', '4.857M', '2.926B', '--', '-62.38%', ''], ['MCRB', 'Seres Therapeutics, Inc.', '', '17.27+1.02(+6.25%)', '+1.02', '+6.25%', '22,100', '119,888', '151.149M', '--', '-7.04%', ''], ['COCO', 'The Vita Coco Company, Inc.', '', '35.99+2.05(+6.04%)', '+2.05', '+6.04%', '424,987', '645,720', '2.045B', '33.64', '+33.89%', ''], ['HTHT', 'H World Group Limited', '', '35.10+1.76(+5.26%)', '+1.76', '+5.26%', '2.476M', '1.763M', '10.775B', '24.55', '+16.94%', ''], ['STEM', 'Stem, Inc.', '', '15.60+

{'missing': [], 'shape': (25, 12), 'na_total': 25}

In [14]:
_ = save_csv(df_scrape, prefix='scrape', site='yahoo_finance', table='gainers')

Saved data/raw/scrape_site-yahoo_finance_table-gainers_20250820-114424.csv


## Documentation & Insights
- API: `https://www.alphavantage.co/query` (TIME_SERIES_DAILY, `AAPL`, `compact`). Free tier enforces strict rate limits; batch downstream processing to avoid retries.
- Scrape: `https://finance.yahoo.com/gainers`. Fields are volatile; treat scraped columns as soft schema and validate per run.
- Persistence: Timestamped filenames under `data/raw/` enable reproducibility and backfills.
- Practical insight: For downstream modeling, join API `close` prices with scraped tickers by trading day. Use scrape only for qualitative features (e.g., intraday momentum proxy) due to schema instability.
